# `PythonJob` to run Python function on a remote computer

`PythonJob` node is a built-in node which allow user to run Python function on a remote computer.


## File Handling

### Remote Folder
List the file in the remote folder:
```console
$ ls
aiida.out        inputs.pickle   _scheduler-stderr.txt  script.py
_aiidasubmit.sh  results.pickle  _scheduler-stdout.txt
```

Each node creates a `script.py` file on the remote computer, which includes:
- The function definition.
- Loading inputs from `inputs.pickle`.
- Running the function with the loaded inputs.
- Saving the results into `results.pickle`.


### About the data
For a `CalcJob`, the input data needs to be an AiiDA data node; however, we don't require the user to install AiiDA or the same Python environment on a remote computer. This means we should pass normal Python data as arguments when running the Python function on the remote computer. The `WorkGraphEngine` will handle this data transformation when preparing and launching the `CalcJob`.

All AiiDA data that will be passed to the function should have a `value` attribute, which corresponds to its raw Python data. The `GeneralData`, `Int`, `Float`, `Str`, `Bool` fulfill this requirement, while `List`, `Dict` and `StructureData` are not.

### Inputs and Outputs:
Inputs for each node are pickled into the `inputs.pickle` file.
Outputs from each node are pickled into the `results.pickle` file.

### Parent Folder
The parent_folder parameter allows a node to access the output files of a parent node. This feature is particularly useful when you want to reuse data generated by a previous computation in subsequent computations. In the provided example, the multiply node uses the result.txt file created by the add node.


## Example

```python
from aiida_workgraph import WorkGraph, node
from aiida import orm, load_profile

load_profile()

# define add node
@node()
def add(x, y):
    return x + y

# define multiply node
@node()
def multiply(x, y):
    return x*y

wg = WorkGraph("first_workflow")
wg.nodes.new(add, name="add", run_remotely=True)
wg.nodes.new(multiply, name="multiply", x=wg.nodes["add"].outputs[0], run_remotely=True)


#------------------------- Submit the calculation -------------------
wg.submit(inputs = {"add": {"x": 2, "y": 3, "computer": "localhost"},
                    "multiply": {"y": 4, "computer": "localhost"}},
          wait=True)
print("\nResult of multiply is {} \n\n".format(wg.nodes["multiply"].outputs['result'].value))
```

